In [ ]:
import jinja2
from ipywidgets import HTML

def create_legend(labels: list) -> HTML:
    """Creates an HTML legend from a list dictionary of the format {'text': str, 'color': [r, g, b]}"""
    labels = list(labels)
    for label in labels:
        assert label['color'] and label['text']
        assert len(label['color']) in (3, 4)
        label['color'] = ', '.join([str(c) for c in label['color']])
    legend_template = jinja2.Template('''
    <style>
      .legend {
        width: 300px;
      }
      .square {
        height: 10px;
        width: 10px;
        border: 1px solid grey;
      }
      .left {
        float: left;
      }
      .right {
        float: right;
      }
    </style>
    {% for label in labels %}
    <div class='legend'>
      <div class="square left" style="background:rgba({{ label['color'] }})"></div>
      <span class="right">{{label['text']}}</span>
      <br />
    </div>
    {% endfor %}
    ''')
    html_str = legend_template.render(labels=labels)
    return HTML(html_str)

In [ ]:
import pydeck as pdk
import pandas as pd

from ast import literal_eval

DATA_URL = 'https://raw.githubusercontent.com/ajduberstein/geo_datasets/master/portugal-taxi-trajectories.csv'
df = pd.read_csv(DATA_URL)
# Assign random colors to each neighborhood category
color_lookup = pdk.data_utils.assign_random_colors(df['CALL_TYPE'])
df['color'] = df['CALL_TYPE'].apply(lambda row: color_lookup.get(row))
# Compute a viewport that fits that middle 90% of the data
view_state = pdk.ViewState(longitude=-8.618643, latitude=41.141412, zoom=10)
df['POLYLINE'] = df['POLYLINE'].apply(lambda x: literal_eval(x))

In [ ]:
layers = [
    pdk.Layer(
        "PathLayer",
        data=df.head(100),
        get_path='POLYLINE',
        get_width=20,
        get_color='color',
        pickable=True,
        auto_highlight=True),
]

In [ ]:
legend_lookup = [{'text': t, 'color': c} for t, c in color_lookup.items()]
legend = create_legend(legend_lookup)

In [ ]:
legend

In [ ]:
display(HTML("""
   <strong>Taxi Trip Type</strong>
   (Data from <a href="https://www.kaggle.com/c/pkdd-15-predict-taxi-service-trajectory-i/data">Kaggle</a>)
"""))
display(legend)
pdk.Deck(
    layers,
    initial_view_state=view_state
).to_html()